In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv('../input/bike-sharing-demand/train.csv')
df_test = pd.read_csv('../input/bike-sharing-demand/test.csv')
df_test_fixed = pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
print(df_train.isnull().sum().sum())
print(df_test.isnull().sum().sum())

print(df_train.duplicated().sum())
print(df_test.duplicated().sum())

In [ ]:
df_train.describe()

In [ ]:
df_train['datetime'] = pd.to_datetime(df_train['datetime'])

df_train["year"] = df_train["datetime"].dt.year
df_train['month'] = df_train["datetime"].dt.month
df_train['day'] = df_train["datetime"].dt.day
df_train["hour"] = df_train["datetime"].dt.hour
df_train["dayofweek"] = df_train["datetime"].dt.dayofweek

df_train.drop('datetime', axis=1, inplace=True)

df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_test['datetime'] = pd.to_datetime(df_test['datetime'])

df_test["year"] = df_test["datetime"].dt.year
df_test['month'] = df_test["datetime"].dt.month
df_test['day'] = df_test["datetime"].dt.day
df_test["hour"] = df_test["datetime"].dt.hour
df_test["dayofweek"] = df_test["datetime"].dt.dayofweek

df_test.drop('datetime', axis=1, inplace=True)

In [ ]:
df_test.info()

# EDA

In [ ]:
sns.countplot(x = 'season', data = df_train);

In [ ]:
sns.countplot(x = 'season', data = df_train);

In [ ]:
sns.countplot(x = 'workingday', data = df_train);

In [ ]:
sns.countplot(x = 'weather', data = df_train);

In [ ]:
sns.countplot(x = 'dayofweek', data = df_train);

In [ ]:
sns.countplot(x = 'hour', data = df_train);

In [ ]:
sns.countplot(x = 'month', data = df_train);

In [ ]:
Columns = ['weather', 'temp','atemp', 'humidity', 'windspeed']
sns.pairplot(df_train, vars = Columns, diag_kind = 'kde');

In [ ]:
X = df_train.drop(['casual', 'registered', 'count'], axis=1)
X.head()

In [ ]:
Y_casual = df_train['casual']
Y_registered = df_train['registered']
Y_count = df_train['count']

In [ ]:
x_train_count, x_valid_count, y_train_count, y_valid_count = train_test_split(X , Y_count,  train_size=0.8, test_size=0.2, random_state =0)
x_train_casual, x_valid_casual, y_train_casual, y_valid_casual = train_test_split(X , Y_casual,  train_size=0.8, test_size=0.2, random_state =0)
x_train_registered, x_valid_registered, y_train_registered, y_valid_registered = train_test_split(X , Y_registered,  train_size=0.8, test_size=0.2, random_state =0)

In [ ]:
# Random forest, count

model_rf_count = RandomForestRegressor(n_estimators=300)
model_rf_count.fit(x_train_count,y_train_count)
y_pred_rf_count = model_rf_count.predict(x_valid_count)
score_rf_count = np.sqrt(mean_squared_error(np.log1p(y_valid_count) , np.log1p(y_pred_rf_count), squared=False))
print(score_rf_count)

In [ ]:
# Random forest, casual

model_rf_casual = RandomForestRegressor(n_estimators=300)
model_rf_casual.fit(x_train_casual, y_train_casual)
y_pred_rf_casual = model_rf_count.predict(x_valid_casual)
# score_rf_casual = np.sqrt(mean_squared_error(np.log1p(y_valid_casual) , np.log1p(y_pred_rf_casual), squared=False))
# print(score_rf_casual)

In [ ]:
# Random forest, registered

model_rf_registered = RandomForestRegressor(n_estimators=300)
model_rf_registered.fit(x_train_registered, y_train_registered)
y_pred_rf_registered = model_rf_count.predict(x_valid_registered)
# score_rf_registered = np.sqrt(mean_squared_error(np.log1p(y_valid_registered) , np.log1p(y_pred_rf_registered), squared=False))
# print(score_rf_registered)

In [ ]:
y_pred_rf_casual_plus_registered = y_pred_rf_casual + y_pred_rf_registered

In [ ]:
score_rf_casual_plus_registered = np.sqrt(mean_squared_error(np.log1p((y_valid_casual+y_valid_registered)) , np.log1p(y_pred_rf_casual_plus_registered), squared=False))
print(score_rf_casual_plus_registered)

In [ ]:
# import lightgbm as lgb
# hyperparameters_1 = { 'colsample_bytree': 0.725,  'learning_rate': 0.013,
#                     'num_leaves': 56, 'reg_alpha': 0.754, 'reg_lambda': 0.071, 
#                     'subsample': 0.523, 'n_estimators': 1093}

# model_lgb_count_1 = lgb.LGBMRegressor(**hyperparameters_1)
# model_lgb_count_1.fit(x_train_count, y_train_count)
# y_pred_lgb_count_1 = model_lgb_count_1.predict(x_valid_count)

# hyperparameters_2 = { 'colsample_bytree': 0.639,  'learning_rate': 0.011,
#                     'num_leaves': 30, 'reg_alpha': 0.351, 'reg_lambda': 0.587,
#                    'subsample': 0.916, 'n_estimators': 2166}
# model_lgb_count_2 = lgb.LGBMRegressor(**hyperparameters_2)
# model_lgb_count_2.fit(x_train_count, y_train_count)
# y_pred_lgb_count_2 = model_lgb_count_2.predict(x_valid_count)

In [ ]:
# score_lgb_count_1 = np.sqrt(mean_squared_error(np.log1p(y_valid_count) , np.log1p(y_pred_lgb_count_1), squared=False))
# print(score_lgb_count_1)

# score_lgb_count_2 = np.sqrt(mean_squared_error(np.log1p(y_valid_count) , np.log1p(y_pred_lgb_count_2), squared=False))
# print(score_lgb_count_2)

In [ ]:
final_predictions = model_rf_count.predict(df_test)


In [ ]:
submission = pd.DataFrame({'datetime': df_test_fixed['datetime'],
                       'count': final_predictions})
submission.to_csv('submission.csv', index=False)